In [6]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
widths = [1]*32
df = pd.read_fwf('digitdata/optdigits-orig_train.txt', widths=widths, header=None, skiprows=lambda x : (x+1) % 33 == 0)
# df
df_Y = pd.read_fwf('digitdata/optdigits-orig_train.txt', widths=[2], header=None, skiprows=lambda x : (x+1) % 33 != 0)

In [8]:
df_trainX = pd.DataFrame(df.values.reshape(2436, -1))
df_trainY = df_Y
df_trainY.columns = ['label']
# df_Y

In [9]:
df = pd.read_fwf('digitdata/optdigits-orig_test.txt', widths=widths, header=None, skiprows=lambda x : (x+1) % 33 == 0)
df_Y = pd.read_fwf('digitdata/optdigits-orig_test.txt', widths=[2], header=None, skiprows=lambda x : (x+1) % 33 != 0)


In [10]:
df_testX = pd.DataFrame(df.values.reshape(444, -1))
df_testY = df_Y
df_testY.columns = ['label']

# Part 2.1

In [11]:
def predict(feature_set, w):
    predicted = np.empty(len(feature_set))
    for idx in range(len(feature_set)):
        arg_max, predicted_class = 0, 0
        feature_vector = feature_set.iloc[idx]

        for c in range(10):
            current = np.dot(feature_vector, w[c])
            if current >= arg_max:
                arg_max, predicted_class = current, c
                
        predicted[idx] = predicted_class
    return predicted

In [12]:
def calculate_acc_conf_mat(actual, prediction):
    y_pred = pd.Series(prediction)
    df_confusion = pd.crosstab(actual, y_pred)

    conf_percent = df_confusion.values / df_confusion.sum(axis=1).values.reshape(-1,1)
    df_conf_percent = round(pd.DataFrame(conf_percent).rename_axis('Actual').rename_axis('Prediction', axis=1), 4)*100

    return np.mean(np.diag(df_conf_percent)), df_conf_percent

In [29]:
# df_trainX.shape
df_train = pd.concat([df_trainX, df_trainY], axis=1)
epoch_num = 3
enumerate(df_train_all)
df_num = 0
df_train.values
# df_train N x 1024, N = 2436
# weight_vectors_zero_rand 
np.array(df_train.iloc[0])

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [39]:
df_train.iloc[1]

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       1
12       1
13       1
14       1
15       1
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
995      0
996      0
997      0
998      0
999      0
1000     0
1001     0
1002     0
1003     1
1004     1
1005     1
1006     1
1007     1
1008     1
1009     1
1010     1
1011     0
1012     0
1013     0
1014     0
1015     0
1016     0
1017     0
1018     0
1019     0
1020     0
1021     0
1022     0
1023     0
label    0
Name: 1, Length: 1025, dtype: int64

In [14]:
df_train_all = [pd.concat([df_trainX, df_trainY], axis=1),         # no bias, fixed order
                pd.concat([df_trainX, df_trainY], axis=1),         # with bias, fixed order
                pd.concat([df_trainX, df_trainY], axis=1),         # no bias, random order
                pd.concat([df_trainX, df_trainY], axis=1)]         # with bias, random order
df_train_all[1].insert(1024, 'bias', 1)

df_train_all[2] = df_train_all[2].reindex(np.random.permutation(df_train_all[1].index))
df_train_all[2] = df_train_all[2].reset_index(drop=True)

df_train_all[3].insert(1024, 'bias', 1)
df_train_all[3] = df_train_all[3].reindex(np.random.permutation(df_train_all[3].index))
df_train_all[3] = df_train_all[3].reset_index(drop=True)

df_test_all = [pd.concat([df_testX, df_testY], axis=1),         # no bias
               pd.concat([df_testX, df_testY], axis=1)]         # with bias
df_test_all[1].insert(1024, 'bias', 1)

epoch_nums = [3, 5, 10]
# for epoch_num in epoch_nums:
epoch_num = epoch_nums[0]      
            

In [ ]:
train

for df_num, df_train in enumerate(df_train_all):
        weight_vectors_zero_rand = [np.zeros((10, 1024)), 
                                    np.zeros((10, 1025)), 
                                    np.random.rand(10, 1024)*10, 
                                    np.random.rand(10, 1025)*10]
        for weight_num, weight_vectors in enumerate(weight_vectors_zero_rand):
            if df_train.values.shape[1] != weight_vectors.shape[1] + 1:  
                continue
            acc_per_epoch = []
            eta = 1
            for epoch in range(1,epoch_num+1):
                for idx in range(len(df_train)):
                    arg_max, predicted_class = 0, 0
                    category = df_train.iloc[idx,-1]
                    feature_vector = df_train.iloc[idx,:-1].values
                    
                    
                    for c in range(10):
                        current = np.dot(feature_vector, weight_vectors[c])
                        if current >= arg_max:
                            arg_max, predicted_class = current, c

                    # update weights 
                    if not (category == predicted_class):
                        weight_vectors[category] += eta * feature_vector
                        weight_vectors[predicted_class] -= eta * feature_vector
                train_pred = predict(df_train.iloc[:,:-1], weight_vectors)
                acc, _ = calculate_acc_conf_mat(df_train.iloc[:,-1], train_pred)
                acc_per_epoch.append(acc)
                eta = eta / epoch

In [ ]:
# # data visulization

# f, ax = plt.subplots(1, 1, figsize=(12,12))
#             ax.plot(range(1,epoch_num+1), acc_per_epoch, marker='.')
#             ax.set_xlabel('Epoch', fontsize=18)
#             ax.set_ylabel('Accuracy', fontsize=18)
#             title_str = ''
#             if weight_num > 1:
#                 title_str += 'Weights initialized to Random'
#                 if weight_vectors.shape[1] == 1025:
#                     title_str += ', with Bias'
#                     if df_num > 1:
#                         title_str += ', Random Ordering of training set'
#                     else:
#                         title_str += ', Fixed Ordering of training set'
#                 else:
#                     title_str += ', no Bias'
#                     if df_num > 1:
#                         title_str += ', Random Ordering of training set'
#                     else:
#                         title_str += ', Fixed Ordering of training set'
#             else:
#                 title_str += 'Weights initialized to Zeros'
#                 if weight_vectors.shape[1] == 1025:
#                     title_str += ', with Bias'
#                     if df_num > 1:
#                         title_str += ', Random Ordering of training set'
#                     else:
#                         title_str += ', Fixed Ordering of training set'
#                 else:
#                     title_str += ', no Bias'
#                     if df_num > 1:
#                         title_str += ', Random Ordering of training set'
#                     else:
#                         title_str += ', Fixed Ordering of training set'
#             plt.title('Training Accuracy vs. Epoch Plot ({})'.format(title_str), fontsize=20)
#             plt.show()

#             if weight_vectors.shape[1] == 1025:
#                 df_to_test = df_test_all[1]
#             else:
#                 df_to_test = df_test_all[0]
#             test_pred = predict(df_to_test.iloc[:,:-1], weight_vectors)
#             acc_test, conf_mat_test = calculate_acc_conf_mat(df_to_test.iloc[:,-1], test_pred)
#             print('Test set Confusion Matrix: Actual vs. Prediction')
#             display(conf_mat_test)
#             display(pd.DataFrame(np.diag(conf_mat_test), columns=['Accuracy']))
#             print('Average Accuracy: {}'.format(acc_test))
#             print('\n')